### Alternating Least Squares


- **ALS (Alternating Least Squares)** là thuật toán được sử dụng để **đề xuất sản phẩm** và **danh mục sản phẩm** phù hợp cho người dùng. Thuật toán này hoạt động bằng cách tối ưu hóa **ma trận tương tác** giữa người dùng và sản phẩm theo từng bước lặp.

- Mỗi bước lặp, ALS cố gắng **tiếp cận** dần với cách biểu diễn được hệ số hóa của dữ liệu gốc. Nói cách khác, thuật toán sẽ phân tích **ma trận tương tác** thành hai ma trận nhỏ hơn, đại diện cho **sở thích** của người dùng và **đặc điểm** của sản phẩm. Sau đó, ALS sử dụng thông tin này để **dự đoán** mức độ yêu thích của người dùng đối với các sản phẩm chưa được tương tác trước đây.

In [ ]:
import pandas as pd
import os
import numpy as np
import random

import random
import pandas as pd
import numpy as np
import pickle
import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import MinMaxScaler

In [ ]:
import gdown

file_id ='1FdhKYX7QfInl0DN6UQGYqYnipbHIqgj6'
output_path = 'Olist_Recommendation_Dataset.csv'
gdown.download(f"https://drive.google.com/uc?id={file_id}", output_path, quiet=True, fuzzy=True)

'Olist_Recommendation_Dataset.csv'

In [ ]:
Olist_db = pd.read_csv('Olist_Recommendation_Dataset.csv')

In [ ]:
Olist_db.columns

Index(['Customer State', 'Review Score', 'Purchase Timestamp', 'Purchase Date',
       'Monetary', 'Customer Segment', 'Marketing Action', 'Customer_ID',
       'Order_ID', 'Product_ID', 'product_name', 'aisle', 'department',
       'Product Category'],
      dtype='object')

In [ ]:
Olist_db

,Customer State,Review Score,Purchase Timestamp,Purchase Date,Monetary,Customer Segment,Marketing Action,Customer_ID,Order_ID,Product_ID,product_name,aisle,department,Product Category
0,RJ,5,2017-09-13 08:59:02,2017-09-13 00:00:00,281.30,Potential Loyalists,Cross Sell Recommendations and Discount coupons,15438,0,8446,Utility Lighter,more household,household,more household
1,GO,5,2017-06-28 11:52:20,2017-06-28 00:00:00,73.86,Lost Customers,Don't spend too much trying to re-acquire,22529,7229,8446,Utility Lighter,more household,household,more household
2,MG,4,2018-05-18 10:25:53,2018-05-18 00:00:00,108.32,Potential Loyalists,Cross Sell Recommendations and Discount coupons,17139,31508,8446,Utility Lighter,more household,household,more household
3,PR,5,2017-08-01 18:38:42,2017-08-01 00:00:00,472.06,Loyal Customers,Loyality programs;Cross Sell,33008,42252,8446,Utility Lighter,more household,household,more household
4,MG,5,2017-08-10 21:48:40,2017-08-10 00:00:00,1415.74,VVIP - Can't Loose Them,No Price Incentives; Offer Limited edition and...,41377,47321,8446,Utility Lighter,more household,household,more household
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100846,RJ,1,2017-12-09 13:51:24,2017-12-09 00:00:00,618.88,Potential Loyalists,Cross Sell Recommendations and Discount coupons,3664,94603,14949,California Pinot Noir Red Wine,missing,missing,Hand Luggage
100847,SP,4,2017-07-13 11:24:43,2017-07-13 00:00:00,157.30,Hibernating - Almost Lost,Aggressive price incentives,10847,87800,4042,Chicken Curry with Seasoned Basmati Rice,frozen meals,frozen,frozen meals
100848,SP,3,2018-07-27 10:44:02,2018-07-27 00:00:00,68.35,Potential Loyalists,Cross Sell Recommendations and Discount coupons,32799,88387,8881,16inch Macbook Pro -2.6GHz Intel Core i7,ice cream ice,frozen,Apple Macbook
100849,SC,5,2018-08-21 11:29:05,2018-08-21 00:00:00,314.32,Potential Loyalists,Cross Sell Recommendations and Discount coupons,40805,90090,13612,Roja Parfumes Luxe 3.4 oz,candy chocolate,snacks,Perfumes


In [ ]:
sorted_df = Olist_db.sort_values(by="Customer_ID")
sorted_df

,Customer State,Review Score,Purchase Timestamp,Purchase Date,Monetary,Customer Segment,Marketing Action,Customer_ID,Order_ID,Product_ID,product_name,aisle,department,Product Category
64290,SP,5,2017-11-22 20:40:31,2017-11-22 00:00:00,136.26,Needs Attention,Price incentives and Limited time offer,0,2333,31330,Plain Naan,tortillas flat bread,bakery,tortillas flat bread
39084,SP,5,2018-03-13 09:45:56,2018-03-13 00:00:00,136.26,Needs Attention,Price incentives and Limited time offer,0,83186,12742,Kids Broad Spectrum 50+ Spray Sunscreen,body lotions soap,personal care,body lotions soap
37798,SC,5,2017-09-13 14:55:27,2017-09-13 00:00:00,583.87,VVIP - Can't Loose Them,No Price Incentives; Offer Limited edition and...,1,65709,10569,Mini Dog Treats,dog food care,pets,dog food care
82283,SC,5,2018-04-26 12:01:31,2018-04-26 00:00:00,583.87,VVIP - Can't Loose Them,No Price Incentives; Offer Limited edition and...,1,24429,24407,"Toasted Sesame Oil, Unrefined",oils vinegars,pantry,oils vinegars
31050,SC,5,2017-08-01 14:34:44,2017-08-01 00:00:00,583.87,VVIP - Can't Loose Them,No Price Incentives; Offer Limited edition and...,1,65161,887,Creamy Supreme Buttercream Frosting,baking supplies decor,pantry,baking supplies decor
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98270,CE,5,2017-12-12 14:15:54,2017-12-12 00:00:00,199.50,Potential Loyalists,Cross Sell Recommendations and Discount coupons,41428,42566,15382,Organic Wild Berry Blend,juice nectars,beverages,juice nectars
33045,CE,5,2018-07-09 09:18:47,2018-07-09 00:00:00,199.50,Potential Loyalists,Cross Sell Recommendations and Discount coupons,41428,20120,3433,Sensitivity Ultra Thin Premium Lubricant Condoms,FitSnack,other,FitSnack
81972,RJ,5,2017-09-08 16:31:08,2017-09-08 00:00:00,45.10,Lost Customers,Don't spend too much trying to re-acquire,41429,39281,9838,Salted Whipped Butter,butter,dairy eggs,butter
31845,BA,4,2017-07-19 12:07:16,2017-07-19 00:00:00,366.08,Needs Attention,Price incentives and Limited time offer,41430,30278,1031,Cold Remedy Orange Cream Ultra RapidMelt,KetoKrate,personal care,KetoKrate


In [ ]:
after_unique = Olist_db.drop_duplicates(subset="Customer_ID", keep="first")
after_unique
after_unique["Customer Segment"].value_counts()

Customer Segment
Potential Loyalists          11492
VVIP - Can't Loose Them       9909
Lost Customers                6856
Loyal Customers               5335
Needs Attention               3735
Hibernating - Almost Lost     2541
Champions Big Spenders        1563
Name: count, dtype: int64

In [ ]:
value_counts = after_unique["Customer Segment"].value_counts()
percentages = value_counts / len(after_unique) * 100

print(percentages)

Customer Segment
Potential Loyalists          27.737684
VVIP - Can't Loose Them      23.916874
Lost Customers               16.547995
Loyal Customers              12.876831
Needs Attention               9.014989
Hibernating - Almost Lost     6.133089
Champions Big Spenders        3.772537
Name: count, dtype: float64


In [ ]:
value_counts = Olist_db["Customer Segment"].value_counts()
percentages = value_counts / len(Olist_db) * 100

print(percentages)

Customer Segment
VVIP - Can't Loose Them      40.070004
Potential Loyalists          19.857017
Loyal Customers              16.694926
Lost Customers                8.241862
Needs Attention               6.757494
Champions Big Spenders        4.532429
Hibernating - Almost Lost     3.846268
Name: count, dtype: float64


In [ ]:
Olist_db["Customer Segment"].unique()

array(['Potential Loyalists', 'Lost Customers', 'Loyal Customers',
       "VVIP - Can't Loose Them", 'Champions Big Spenders',
       'Needs Attention', 'Hibernating - Almost Lost'], dtype=object)

In [ ]:
features = ['Customer_ID', 'product_name', 'Product_ID', 'Review Score']
Olist_db_ = Olist_db[features]
Olist_db_ = Olist_db_.rename(columns={'Review Score': 'Ratings'})
Olist_db_ = Olist_db_.drop_duplicates()

In [ ]:
from scipy.sparse import csr_matrix

In [ ]:
Olist_db_ = Olist_db_[:20000]

In [ ]:
Olist_db_.Customer_ID.unique()

array([15438, 22529, 17139, ...,  6650, 27126, 18524])

Xoay DataFrame Olist_db_ để tạo ma trận xếp hạng sản phẩm của người dùng, điền các giá trị còn thiếu bằng 0, chuyển đổi thành ma trận thưa thớt

In [ ]:
df_product_features = Olist_db_.reset_index().pivot_table(
    index='Customer_ID',
    columns='Product_ID',
    values='Ratings'
).fillna(0)

mat_product_features = csr_matrix(df_product_features.values)

In [ ]:
df_product_features.head()

Product_ID,63,69,151,161,185,219,226,238,255,311,...,32021,32050,32052,32083,32115,32134,32164,32170,32171,32203
Customer_ID,,,,,,,,,,,,,,,,,,,,,
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
df_product_features.shape

(15534, 1172)

In [ ]:
X = df_product_features.T
X

Customer_ID,5,9,12,15,16,19,20,24,27,29,...,41410,41412,41413,41416,41418,41419,41421,41422,41424,41425
Product_ID,,,,,,,,,,,,,,,,,,,,,
63,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
69,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
151,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
161,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
185,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32134,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
32164,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
32170,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
X.index[:-20]

Index([   63,    69,   151,   161,   185,   219,   226,   238,   255,   311,
       ...
       31567, 31575, 31585, 31605, 31607, 31631, 31654, 31668, 31671, 31692],
      dtype='int64', name='Product_ID', length=1152)

In [ ]:
# Khởi tạo một đối tượng TruncatedSVD với n_components được đặt thành 10.
# TruncatedSVD là một kỹ thuật giảm kích thước thường được sử dụng cho dữ liệu thưa thớt.
SVD = TruncatedSVD(n_components=10)

# Khớp mô hình TruncatedSVD với ma trận dữ liệu X và biến nó thành biểu diễn có chiều thấp hơn.
decomposed_matrix = SVD.fit_transform(X)
decomposed_matrix.shape
pd.DataFrame(decomposed_matrix)

,0,1,2,3,4,5,6,7,8,9
0,0.004683,5.190728e-04,0.001411,0.002910,-0.000777,0.001395,-0.007554,-0.003517,0.006398,0.013871
1,0.000004,-1.308879e-08,0.000008,0.000012,0.000003,-0.000007,-0.000007,0.000011,0.000025,-0.000018
2,0.007440,2.216847e-03,0.014899,0.006140,0.021616,-0.005470,0.048099,-0.002841,0.022663,0.385810
3,0.060661,2.682763e-02,0.324646,0.337669,0.150943,-0.125355,0.520280,0.111028,-0.247316,-0.078955
4,0.018820,3.839452e-03,0.274012,0.288154,-0.136399,-0.071266,-0.030852,0.010909,-0.040351,-0.000320
...,...,...,...,...,...,...,...,...,...,...
1167,0.001057,9.109338e-04,0.000010,0.000337,0.001282,0.000337,-0.002548,0.000300,-0.000823,0.004450
1168,0.111033,2.047371e-01,-0.012204,-0.000138,-0.023376,0.006226,-0.015795,-0.002425,-0.015177,0.003882
1169,0.002844,2.244552e-03,0.002963,0.001603,0.004482,0.000660,0.007033,-0.001149,0.000020,0.007942
1170,0.002157,4.679233e-04,0.002430,0.001711,0.001854,-0.002762,0.000303,0.001258,-0.011220,-0.002767


In [ ]:
# Tính toán ma trận tương quan từ ma trận phân rã bằng hàm corrcoef.
# Ma trận tương quan cho thấy mối tương quan giữa các cặp biến (trong trường hợp này là features) trong ma trận phân rã.
correlation_matrix = np.corrcoef(decomposed_matrix)
correlation_matrix.shape

(1172, 1172)

In [ ]:
pd.DataFrame(correlation_matrix)

,0,1,2,3,4,5,6,7,8,9,...,1162,1163,1164,1165,1166,1167,1168,1169,1170,1171
0,1.000000,-0.128304,0.682731,-0.625814,0.088515,0.222447,0.358136,-0.100254,-0.333137,0.112344,...,0.425364,-0.659156,0.064660,-0.452568,-0.813260,0.765204,0.058298,0.174944,-0.419417,0.643752
1,-0.128304,1.000000,-0.617297,-0.101135,0.264343,-0.301322,0.284962,0.174561,0.536807,0.307577,...,0.577157,-0.231493,0.087430,-0.359179,0.063204,-0.447471,-0.122246,-0.727685,-0.287870,-0.690803
2,0.682731,-0.617297,1.000000,-0.215061,-0.100193,0.330867,0.146743,-0.018133,-0.585613,-0.150935,...,0.198165,-0.319535,0.016008,-0.202966,-0.485827,0.712253,-0.150686,0.685811,-0.193777,0.724807
3,-0.625814,-0.101135,-0.215061,1.000000,0.464572,0.273599,0.212571,0.349198,0.324255,-0.076735,...,-0.403709,0.890770,0.271323,0.649731,0.846096,-0.444254,-0.185406,0.345787,0.698332,-0.199872
4,0.088515,0.264343,-0.100193,0.464572,1.000000,0.134953,0.551339,0.014695,0.495251,-0.153127,...,-0.022202,0.256203,0.637739,0.219877,0.127459,-0.058698,-0.052125,-0.126065,0.347136,-0.057411
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1167,0.765204,-0.447471,0.712253,-0.444254,-0.058698,0.607433,0.366955,0.154802,-0.448760,-0.173384,...,-0.045366,-0.293134,-0.008471,0.072542,-0.768675,1.000000,0.178171,0.289331,0.072148,0.809465
1168,0.058298,-0.122246,-0.150686,-0.185406,-0.052125,0.147753,-0.015705,-0.017723,-0.264722,-0.137874,...,-0.262176,0.105053,-0.008005,0.227434,-0.088809,0.178171,1.000000,-0.090267,0.204890,0.060894
1169,0.174944,-0.727685,0.685811,0.345787,-0.126065,0.502055,0.124891,0.290852,-0.355111,0.002307,...,-0.128868,0.312474,0.025072,0.188905,0.211942,0.289331,-0.090267,1.000000,0.191188,0.514418
1170,-0.419417,-0.287870,-0.193777,0.698332,0.347136,0.607656,0.256669,0.415064,0.184487,-0.130727,...,-0.746411,0.884395,0.263933,0.984928,0.418594,0.072148,0.204890,0.191188,1.000000,0.187646


In [ ]:
fave_prod = Olist_db_.groupby(['Customer_ID']).max()['Product_ID'].to_frame()

In [ ]:
fave_prod = fave_prod.reset_index()
fave_prod

,Customer_ID,Product_ID
0,5,10630
1,9,6927
2,12,15675
3,15,27919
4,16,10630
...,...,...
15529,41419,13331
15530,41421,18573
15531,41422,15250
15532,41424,21641


In [ ]:
def get_prod_id(customer_id):

    product_id = fave_prod[fave_prod.Customer_ID == customer_id]['Product_ID']
    return product_id


prd_id = get_prod_id(41422)
Product_id = prd_id.iloc[0]
Product_id

15250

In [ ]:
product_names = list(X.index)
product_ID_Idx = product_names.index(Product_id)
product_ID_Idx

531

In [ ]:
correlation_product_ID = correlation_matrix[product_ID_Idx]
correlation_product_ID

array([ 0.00727573, -0.62056314,  0.61130876, ...,  0.54468914,
        0.38519966,  0.47053249])

### Recommending top 10 highly coorelated products in sequence

In [ ]:
Recommend = list(X.index[correlation_product_ID > 0.70])

# Removes the item already bought by the customer
Recommend.remove(Product_id)
Recommend[0:20]

[1477, 3196, 3417, 12089, 16929, 16961, 16963, 20280, 23479, 28058]

In [ ]:
## Getting Product names from prediction

# Tạo dự đoán chứa 20 chỉ mục sản phẩm được đề xuất đầu tiên.
predictions = pd.DataFrame(Recommend[:20])
predictions.columns = ['Product_ID']
predictions

,Product_ID
0,1477
1,3196
2,3417
3,12089
4,16929
5,16961
6,16963
7,20280
8,23479
9,28058


In [ ]:
predictions['Product Name'] = predictions.Product_ID.apply(lambda x : Olist_db_[Olist_db_.Product_ID == x]['product_name'].unique()[0])

In [ ]:
predictions[:10]

,Product_ID,Product Name
0,1477,Wild Mushroom Cauliflower Hempseed Burgers
1,3196,Healthy Volume Shampoo
2,3417,Chocolate Creme Cookies
3,12089,Citrus + Mint Soap
4,16929,Dried Shiitake Mushrooms
5,16961,45 Calories & Delightful Healthy Multi-Grain B...
6,16963,Mint Umcka Cold Care Chewable Tablets
7,20280,Jumbo Lump Crab Cake
8,23479,Classic Vegetables Crinkle Cut Carrots
9,28058,Gluten Free Sliced Mountain White Bread


In [ ]:
Olist_db_.to_csv('Olist_db_ALS.csv', index=False)

In [ ]:
X.to_csv('X_ALS.csv')

In [ ]:
correlation_matrix

array([[ 1.        , -0.1283044 ,  0.68273084, ...,  0.17494379,
        -0.41941736,  0.64375216],
       [-0.1283044 ,  1.        , -0.61729684, ..., -0.7276852 ,
        -0.28787018, -0.6908029 ],
       [ 0.68273084, -0.61729684,  1.        , ...,  0.68581054,
        -0.1937767 ,  0.72480687],
       ...,
       [ 0.17494379, -0.7276852 ,  0.68581054, ...,  1.        ,
         0.19118791,  0.51441819],
       [-0.41941736, -0.28787018, -0.1937767 , ...,  0.19118791,
         1.        ,  0.18764574],
       [ 0.64375216, -0.6908029 ,  0.72480687, ...,  0.51441819,
         0.18764574,  1.        ]])

In [ ]:
correlation_matrix.shape

(1172, 1172)

In [ ]:
import json

with open('correlation_matrix_ALS.txt', 'w') as filehandle:
    json.dump(correlation_matrix.tolist(), filehandle)

### Prediction Pipeline

In [ ]:
### use 3 files to predict for a User

In [ ]:
Olist_db_ = pd.read_csv('Olist_db_ALS.csv')

In [ ]:
X = pd.read_csv('X_ALS.csv', index_col=0)
X

,5,9,12,15,16,19,20,24,27,29,...,41410,41412,41413,41416,41418,41419,41421,41422,41424,41425
Product_ID,,,,,,,,,,,,,,,,,,,,,
63,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
69,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
151,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
161,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
185,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32134,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
32164,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
32170,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
X.columns[4000:4020]

Index(['10670', '10671', '10675', '10677', '10679', '10680', '10681', '10682',
       '10685', '10686', '10689', '10694', '10696', '10698', '10699', '10700',
       '10703', '10712', '10713', '10714'],
      dtype='object')

In [ ]:
import json
with open('correlation_matrix_ALS.txt') as f:
    correlation_matrix = json.load(f)

In [ ]:
correlation_matrix = np.array(correlation_matrix)

In [ ]:
X.index

Index([   63,    69,   151,   161,   185,   219,   226,   238,   255,   311,
       ...
       32021, 32050, 32052, 32083, 32115, 32134, 32164, 32170, 32171, 32203],
      dtype='int64', name='Product_ID', length=1172)

### Streamlit

In [ ]:
def Recommendations_ALS(Customer_id):
    fave_prod = Olist_db_.groupby(['Customer_ID']).max()['Product_ID'].to_frame()
    fave_prod = fave_prod.reset_index()

    #prd_id = get_prod_id(Customer_id)
    prd_id = fave_prod[fave_prod.Customer_ID == Customer_id]['Product_ID']
    Product_id = prd_id.iloc[0]
    product_names = list(X.index)
    product_ID_Idx = product_names.index(Product_id)

    correlation_product_ID = correlation_matrix[product_ID_Idx]

    Recommend = list(X.index[correlation_product_ID > 0.70])
    # Removes the item already bought by the customer
    Recommend.remove(Product_id)

    ## Getting Product names froom prediction

    predictions = pd.DataFrame(Recommend[:20])
    predictions.columns = ['Product_ID']

    predictions['Product Name'] = predictions.Product_ID.apply(lambda x : Olist_db_[Olist_db_.Product_ID == x]['product_name'].unique()[0])
    Recommendations = predictions[:10]
    return Recommendations

In [ ]:
Recommendations = Recommendations_ALS(10679)
Recommendations[:10000]

,Product_ID,Product Name
0,185,Chicken Nugget Meal
1,504,Velveeta Shells & Cheese Bold Jalapeno
2,2694,Superberry Kombucha
3,3947,16inch Macbook Pro -2.6GHz Intel Core i7
4,4575,"Beef Jerky, Teriyaki"
5,4621,Organic Granny Smith Apple Bag
6,5140,Lean & Fit Coconut Lemongrass Chicken
7,6367,Sugar Free Concord Grape Jam
8,6488,Grape Water Enhancer
9,7006,Original Alcohol Free Witch Hazel With Aloe Vera


In [ ]:
import pandas as pd
import numpy as np
import pickle
from scipy.sparse import csr_matrix
from sklearn.decomposition import TruncatedSVD
import json

# Function to load and prepare data
def load_and_prepare_data(olist_db_path, x_als_path, correlation_matrix_path):
    """Loads and prepares the data for the recommendation model.

    Args:
        olist_db_path (str): Path to the Olist_db_ALS.csv file.
        x_als_path (str): Path to the X_ALS.csv file.
        correlation_matrix_path (str): Path to the correlation_matrix_ALS.txt file.

    Returns:
        tuple: A tuple containing the following:
            - Olist_db_ (pd.DataFrame): The Olist dataset with customer and product information.
            - X (pd.DataFrame): The product features matrix.
            - correlation_matrix (np.ndarray): The correlation matrix between products.
    """
    Olist_db_ = pd.read_csv(olist_db_path)
    X = pd.read_csv(x_als_path, index_col=0)
    with open(correlation_matrix_path) as f:
        correlation_matrix = json.load(f)
    correlation_matrix = np.array(correlation_matrix)
    return Olist_db_, X, correlation_matrix

# Function to get recommendations
def recommendations_als(customer_id, olist_db_, x, correlation_matrix, num_recommendations=10):
    """Generates product recommendations for a given customer using ALS.

    Args:
        customer_id (int): The ID of the customer.
        olist_db_ (pd.DataFrame): The Olist dataset with customer and product information.
        x (pd.DataFrame): The product features matrix.
        correlation_matrix (np.ndarray): The correlation matrix between products.
        num_recommendations (int, optional): The number of recommendations to generate. Defaults to 10.

    Returns:
        pd.DataFrame: A DataFrame containing the recommended products and their names.
    """
    fave_prod = olist_db_.groupby(['Customer_ID']).max()['Product_ID'].to_frame()
    fave_prod = fave_prod.reset_index()
    prd_id = fave_prod[fave_prod.Customer_ID == customer_id]['Product_ID']
    product_id = prd_id.iloc[0]
    product_names = list(X.index)
    product_id_idx = product_names.index(product_id)
    correlation_product_id = correlation_matrix[product_id_idx]
    recommend = list(X.index[correlation_product_id > 0.70])
    recommend.remove(product_id)  # Remove already bought product
    predictions = pd.DataFrame(recommend[:20])  # Get top 20 candidates
    predictions.columns = ['Product_ID']
    predictions['Product Name'] = predictions.Product_ID.apply(lambda x: olist_db_[olist_db_.Product_ID == x]['product_name'].unique()[0])
    recommendations = predictions[:num_recommendations]  # Select the desired number of recommendations
    return recommendations

# Function to save the model
def save_model(olist_db_path, x_als_path, correlation_matrix_path, filename):
    """Saves the recommendation model to a pickle file.

    Args:
        olist_db_path (str): Path to the Olist_db_ALS.csv file.
        x_als_path (str): Path to the X_ALS.csv file.
        correlation_matrix_path (str): Path to the correlation_matrix_ALS.txt file.
        filename (str): The name of the pickle file to save the model to.
    """
    olist_db_, x, correlation_matrix = load_and_prepare_data(olist_db_path, x_als_path, correlation_matrix_path)
    model = {
        'olist_db_': olist_db_,
        'x': x,
        'correlation_matrix': correlation_matrix,
        'recommendations_als': recommendations_als
    }
    with open(filename, 'wb') as f:
        pickle.dump(model, f)

# Example usage (assuming data files are in the same directory)
if __name__ == '__main__':
    olist_db_path = 'Olist_db_ALS.csv'
    x_als_path = 'X_ALS.csv'
    correlation_matrix_path = 'correlation_matrix_ALS.txt'
    filename = 'SVD_ALS_model.pkl'
    save_model(olist_db_path, x_als_path, correlation_matrix_path, filename)

In [ ]:
import pickle

# Load the model
with open('SVD_ALS_model.pkl', 'rb') as f:
    model = pickle.load(f)

# Get recommendations for a customer (e.g., customer ID 10679)
recommendations = model['recommendations_als'](10679, model['olist_db_'], model['x'], model['correlation_matrix'], num_recommendations=15)

# Print the recommendations
print(recommendations)

    Product_ID                                      Product Name
0          185                               Chicken Nugget Meal
1          504            Velveeta Shells & Cheese Bold Jalapeno
2         2694                               Superberry Kombucha
3         3947          16inch Macbook Pro -2.6GHz Intel Core i7
4         4575                              Beef Jerky, Teriyaki
5         4621                    Organic Granny Smith Apple Bag
6         5140             Lean & Fit Coconut Lemongrass Chicken
7         6367                      Sugar Free Concord Grape Jam
8         6488                              Grape Water Enhancer
9         7006  Original Alcohol Free Witch Hazel With Aloe Vera
10        8108                             Spaghetti Style Pasta
11        8721                                 Lite Rice Vinegar
12        8812    Tidy Cats Clumping Litter 4-in-1 Strength Size
13        9442                                  Charentais Melon
14        9510           